In [ ]:
import random
from collections import defaultdict, namedtuple

# Define a namedtuple for classes
ClassInfo = namedtuple('ClassInfo', ['periods', 'teacher', 'students'])
# Define the maximum number of iterations and other ACO parameters
MAX_ITERATIONS = 50
NUM_ANTS = 10
EVAPORATION_RATE = 0.5

def read_input():
    # Read N and M from the keyboard
    N, M = map(int, input().split())

    # Read class info
    classes = []
    print()
    for _ in range(N):
        periods, teacher, students = map(int, input().split())
        classes.append(ClassInfo(periods, teacher, students))

    # Read room capacities
    room_capacities = list(map(int, input().split()))

    return N, M, classes, room_capacities

def initialize_pheromones(N, M, periods):
    # Initialize pheromone levels to 1.0 for all possible (class, period, room) combinations
    pheromones = defaultdict(lambda: 1.0)
    return pheromones

def select_next_move(pheromones, possible_moves):
    # Calculate probabilities based on pheromones and heuristics
    probabilities = []
    for move in possible_moves:
        pheromone = pheromones[move]
        probabilities.append(pheromone)

    total = sum(probabilities)
    probabilities = [p / total for p in probabilities]

    # Roulette wheel selection
    r = random.uniform(0, 1)
    cumulative_probability = 0.0
    for i, prob in enumerate(probabilities):
        cumulative_probability += prob
        if cumulative_probability >= r:
            return possible_moves[i]

    return possible_moves[-1]

def construct_solution(pheromones, N, M, classes, room_capacities):
    solution = []
    remaining_classes = list(range(N))
    constraints = {
        'teacher': defaultdict(set),
        'room': defaultdict(set)
    }

    while remaining_classes:
        class_idx = remaining_classes.pop(0)
        class_info = classes[class_idx]

        possible_moves = []
        for period in range(60):
            for room in range(M):
                if class_info.students <= room_capacities[room]:
                  teacher_check, room_check = True, True
                  for p in range(period, period + classes[class_idx].periods):
                    if p in constraints['teacher'][class_info.teacher]:
                      teacher_check = False
                      break
                    if p in constraints['room'][room]:
                      room_check = False
                      break
                  if teacher_check and room_check:
                    possible_moves.append((class_idx, period, room))

        if not possible_moves:
            continue

        move = select_next_move(pheromones, possible_moves)
        solution.append(move)

        class_idx, period, room = move
        for p in range(period, period + classes[class_idx].periods):
            constraints['teacher'][classes[class_idx].teacher].add(p)
            constraints['room'][room].add(p)

    return solution

def update_pheromones(pheromones, solutions):
    # Evaporate pheromones
    for key in pheromones.keys():
        pheromones[key] *= (1 - EVAPORATION_RATE)

    # Increase pheromones for the best solutions
    for solution in solutions:
        quality = len(solution)
        for move in solution:
            pheromones[move] += quality

def evaluate_solution(solution):
    return len(solution)

def aco_schedule_classes(N, M, classes, room_capacities):
    pheromones = initialize_pheromones(N, M, 60)

    best_solution = None
    best_quality = -1

    for iteration in range(MAX_ITERATIONS):
        solutions = []

        for _ in range(NUM_ANTS):
            solution = construct_solution(pheromones, N, M, classes, room_capacities)
            solutions.append(solution)

            quality = evaluate_solution(solution)
            if quality > best_quality:
                best_quality = quality
                best_solution = solution

        update_pheromones(pheromones, solutions)

    return best_solution

def main():
    N, M, classes, room_capacities = read_input()
    best_schedule = aco_schedule_classes(N, M, classes, room_capacities)

    print(len(best_schedule))
    for class_idx, period, room in best_schedule:
        print(class_idx + 1, period + 1, room + 1)

if __name__ == "__main__":
    main()